In [94]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from sodapy import Socrata

"""
This notebook is designed to develop the relationship between mortality data due to cardiovascular desease
and defibrilator placements (medical and non-medical). Of course, AED will be used before death, but this data
could account for potential areas that have a higher degree of mortality due to cardiovascular desease. 
Analysis will be done considering comarcal geographical distributions, which is the highest resolution data
we have related to mortality rates.
Of course, AEDs can be used in other medical related deseases, but we will specially focus on cardiovascular 
due to the direct relationship.
"""



"""
The first step is to load the available data into the df and clean it. In dades.ipnyb the mortality data, 
defibrillator and hospital data was cleaned.
"""
client = Socrata("analisi.transparenciacatalunya.cat", None)
desfibriladors = client.get_all("wpud-ukyg")
hospitals = client.get_all("8gmd-gz7i")
df_desfibriladors = pd.DataFrame.from_records(desfibriladors) 
df_hospitals = pd.DataFrame.from_records(hospitals)

df_hospitals = df_hospitals[df_hospitals["categoria"].str.contains(r"Salut\|Centres", na=False)]
df_hospitals = df_hospitals.drop_duplicates(subset='idequipament', keep='first')


df_desfibriladors = df_desfibriladors.drop_duplicates(subset='numero_serie', keep='first')


url = "https://www.idescat.cat/serveis/consultes/ca/censph_10_mun_2024.csv"
df_cens = pd.read_csv(url, sep=";", encoding="utf-8")


df_mortality = pd.read_csv("./Data/mortality.csv", sep=",", encoding="utf-8-sig")


/var/folders/_r/lv31y_xx5hzfyhbw22ddz7gr0000gn/T/ipykernel_82434/1405617251.py:36: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cens = pd.read_csv(url, sep=";", encoding="utf-8")


In [95]:
"""
We read both geopandas documents and normalise the names and change some column names so we work with
the same identifiers all around.
"""
import unicodedata

"""
This makes all the names similar so it is easier to compare them
"""
def normalize(text):
    if isinstance(text, str):
        text = text.lower().strip()
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        text = text.replace('-', ' ').replace("'", "").replace("’", "")
        return text
    return text

data_dir = "../carto/"

municipis = gpd.read_file(data_dir + "municipis.json")
municipis = municipis.rename(columns={'CODIMUNI': 'codi_municipi', 'NOMMUNI':'municipi', 'NOMCOMAR': 'comarca'})
municipis = municipis[['codi_municipi', 'municipi', 'comarca', 'geometry']]
municipis

comarques = gpd.read_file(data_dir + "comarca.json")

comarques = comarques[["NOMCOMAR","geometry"]].rename(columns={"NOMCOMAR":"comarca"})
comarques['comarca'] = comarques['comarca'].apply(normalize)

comarca_map = pd.read_csv("./Data/mpiscatalunya.csv", sep=";", encoding="utf-8")
comarca_map = comarca_map.rename(columns={'Codi': 'codi_municipi', 'Nom':'municipi',
                                          'Nom comarca': 'comarca'})


In [96]:
"""
Now we want to count how many hospitals/AED we have per comarca, and create a df that contains these values
and merge it inot a gpd to have the correct idea.
We also add a column with the mortality per 1000 inh for men/women/total due to cardivascular desease
for each of the comarques.
"""

df_hospi = df_hospitals[['codi_municipi']]
df_desfi = df_desfibriladors[['codi_municipi']]

defi_count = df_desfi.groupby('codi_municipi').size().reset_index(name='num_defi')
hospi_count = df_hospi.groupby('codi_municipi').size().reset_index(name='num_hospi')


In [97]:
"""
Normalise municipal data to make sure all have the same codes
It fixes the municipal codes correctly between datasets.
"""
def fix_code(x):
    x = str(x)  # ensure string
    # Add leading zero if first digit is 8 and length < 6
    if x.startswith('8') and len(x) < 6:
        x = '0' + x
    # Truncate last digit if length is 6
    if len(x) == 6:
        x = x[:-1]
    return x


defi_count['codi_municipi'] = defi_count['codi_municipi'].apply(fix_code)
hospi_count['codi_municipi'] = hospi_count['codi_municipi'].apply(fix_code)
municipis['codi_municipi'] = municipis['codi_municipi'].apply(fix_code)
comarca_map['codi_municipi'] = comarca_map['codi_municipi'].apply(fix_code)

# Filter rows where edat == "total"
df_cens_total = df_cens[(df_cens["edat"] == "total") & (df_cens["sexe"] == "total")].copy()
#we avoid taking the whole population of catalunya
df_cens_total = df_cens_total[df_cens_total["valor"] <= 5000000]
df_cens_total = df_cens_total.reset_index(drop=True)
df_cens_total = df_cens_total.rename(columns={"valor": "pob"})

# Keep only geo name and population
df_cens_pop = df_cens_total[["geo", "pob"]].copy()

df_cens_pop = df_cens_pop.rename(columns={'geo': 'codi_municipi'})
df_cens_pop['codi_municipi'] = df_cens_pop['codi_municipi'].apply(fix_code)



In [98]:
import numpy as np
from shapely.validation import make_valid
"""
After all the data cleaning, we finally got to where we want to be, a data set that has municipi and comarca and the amount of
defibrillators and medical centers in each of these places.

Also, we sum the polygons for each of our comarcas (2022) because the mortality data is for that year
and lluçanes did not exist. This allows us to treat our data uniformly.
"""

total = municipis.merge(defi_count, on='codi_municipi', how='left')
total = total.merge(hospi_count, on='codi_municipi', how='left')
total = total.merge(df_cens_pop, on='codi_municipi', how='left')

total = total.fillna(0)
total = total.replace([np.inf, -np.inf], 0)
total = total.drop(columns=['comarca'])

total = total.merge(
    comarca_map[['codi_municipi', 'comarca']],
    on='codi_municipi',
    how='left'
)

total['geometry'] = total['geometry'].apply(make_valid)
total = total.set_crs("EPSG:4326", allow_override=True)

comarca_totals = total.groupby('comarca')[['num_defi', 'num_hospi', 'pob']].sum().reset_index()

print(total.is_valid.value_counts())

gdf_comarca = total.dissolve(by='comarca', as_index=False)
gdf_comarca = gdf_comarca[['comarca', 'geometry']]
comarca_totals = comarca_totals.merge(gdf_comarca, on='comarca', how='left')
comarca_totals = gpd.GeoDataFrame(
    comarca_totals,
    geometry='geometry',  # tell GeoPandas which column is geometry
    crs=gdf_comarca.crs  # use the same CRS as your dissolved gdf
)


numeric_cols = ['num_defi', 'num_hospi', 'pob']

for col in numeric_cols:
    comarca_totals[col] = pd.to_numeric(comarca_totals[col], errors='coerce').fillna(0)

comarca_totals

True    947
Name: count, dtype: int64


,comarca,num_defi,num_hospi,pob,geometry
0,Alt Camp,64.0,17.0,46388,"POLYGON ((1.23383 41.20393, 1.23383 41.20472, ..."
1,Alt Empordà,283.0,49.0,148732,"MULTIPOLYGON (((2.997 42.13316, 2.99723 42.132..."
2,Alt Penedès,236.0,22.0,114189,"POLYGON ((1.69316 41.2768, 1.69315 41.27542, 1..."
3,Alt Urgell,53.0,21.0,21128,"MULTIPOLYGON (((1.32159 41.98664, 1.32235 41.9..."
4,Alta Ribagorça,15.0,5.0,4040,"POLYGON ((0.85715 42.45393, 0.87114 42.44601, ..."
5,Anoia,443.0,24.0,128432,"POLYGON ((1.60696 41.51295, 1.60717 41.51292, ..."
6,Aran,31.0,8.0,10545,"POLYGON ((0.68006 42.72327, 0.68002 42.72329, ..."
7,Bages,257.0,35.0,185352,"POLYGON ((1.81103 41.60503, 1.81002 41.6048, 1..."
8,Baix Camp,218.0,36.0,204458,"MULTIPOLYGON (((0.93153 40.99126, 0.93134 40.9..."
9,Baix Ebre,134.0,19.0,82399,"MULTIPOLYGON (((0.44356 40.73713, 0.44377 40.7..."


In [99]:
"""
Now comes the cleaning and doing with the medical data. we can select a df when choosing one of 
keys, this allows us to save the whole cleaned df in a compact way. We select one of the death
causes, like Circ (circulatory problems) or Tot (Total). Since these values are death rates, we
should multiply them by the probability that a AED has been used...

From data in OHSCAR, we have that cardiac arrests constitute 45.8/100,000 inhabitants for 2021 in
Catalunya. This means, that a rough estimate is that comarca_pob * cardiac_arrest gives us the
desired data. We can compare this with Circ or other death causes. These cardiac arrests values are
out of hospital cardiac arrests, so work better for DEAs. 
To make it more comarcal-like, assume that every cardiac arrest would result in death if untreated,
then we calculate the percentatge of cardiac_arrests/total_circ_death, which gives us a % where DEAS 
can be used. Then, we scale to each comarcal circ_death.
"""
df_cardio = df_mortality[df_mortality['Cause'].str.contains('Circ', case=False, na=False)]

sexes = ["Total", "Dones", "Homes"]

dfs = {}
for name in sexes:
    df = (df_cardio[df_cardio['Sexe'] == name]
          .drop(columns=["Sexe", "Cause"])
          .rename(columns={'Comarca':'comarca'})
          .reset_index(drop=True))
    df['comarca'] = df['comarca'].apply(normalize)
    dfs[name] = df

mortality_totals = dfs["Total"]
mortality_women  = dfs["Dones"]
mortality_men    = dfs["Homes"]

mortality_totals

,comarca,< 1,1-4,5-14,15-24,25-34,35-44,45-54,55-64,65-74,...,5-14 rel,15-24 rel,25-34 rel,35-44 rel,45-54 rel,55-64 rel,65-74 rel,75-84 rel,>84 rel,Total rel
0,alt camp,0,0,0,0,1,0,1,6,12,...,0.00,0.00,20.36,0.00,13.40,91.66,257.12,756.83,3330.92,192.59
1,alt emporda,0,0,0,0,0,1,8,14,36,...,0.00,0.00,0.00,4.77,33.98,70.64,246.74,931.95,3737.69,211.23
2,alt penedes,0,0,1,0,0,0,6,13,22,...,7.82,0.00,0.00,0.00,30.19,85.89,202.84,711.22,3836.63,188.78
3,alt urgell,0,0,0,0,0,0,2,6,5,...,0.00,0.00,0.00,0.00,60.08,192.12,215.70,689.18,5079.01,325.53
4,alta ribagorca,0,0,0,0,0,0,1,1,0,...,0.00,0.00,0.00,0.00,136.05,167.50,0.00,448.43,2325.58,173.70
5,anoia,0,0,0,0,0,3,7,10,32,...,0.00,0.00,0.00,16.88,31.97,60.24,260.54,876.85,3874.73,208.44
6,bages,0,0,0,0,0,1,10,30,45,...,0.00,0.00,0.00,3.99,33.30,120.87,236.31,754.10,4440.50,259.34
7,baix camp,0,0,0,0,2,2,11,28,57,...,0.00,0.00,9.14,7.06,32.74,104.13,282.47,694.44,3391.05,183.19
8,baix ebre,0,0,0,0,0,1,4,18,16,...,0.00,0.00,0.00,8.94,31.19,161.16,180.40,967.90,4233.30,285.97
9,baix emporda,0,0,0,0,0,2,1,20,32,...,0.00,0.00,0.00,10.40,4.31,97.75,206.83,753.80,4614.66,226.84


In [100]:
"""
We compute the data but now based on comarca and we eliminate llucanes comarca, because it is newer
than older df. We have to take into account that this comarca is small.

Then, we compute the cardiac which essetially is the percentatge of circ deaths per comarca that 
could be caused by a cardia arrest. if we take into account that 45.8/100,000 cases in catalunya,
this is approx: 8.012.231 population gives 3.670 cardiac arrest, which represent a 22.38% of total
circ deaths (16.400). We take this value to calcualte cardiac rel
"""
comarca_totals['comarca'] = comarca_totals['comarca'].apply(normalize)


mortality_dfs = {
    "total": mortality_totals,
    "women": mortality_women,
    "men": mortality_men
}

w_a = {'1-13': 0.0037, '13-50': 0.124, '50-75': 0.404, '75+': 0.469}
age_map = {
    '< 1':'1-13','1-4':'1-13','5-14':'1-13','15-24':'13-50','25-34':'13-50',
    '35-44':'13-50','45-54':'50-75','55-64':'50-75','65-74':'50-75',
    '75-84':'75+','>84':'75+'
}

df_comarques = {}

for key, df_mort in mortality_dfs.items():
    df_comarques[key] = comarca_totals.merge(
        df_mort,
        on='comarca',
        how='left'
    )
    #Raw computation of OHCA based only on total cardiac mortality and no age factors.
    df_comarques[key]["cardiac rel"] = df_comarques[key]["Total rel"]*(3670/16400)
    
    # Multiply each age group by OHCA weight, This gives an edge weighted OHCA
    df = df_comarques[key].copy()
    ohca_signal = []
    for age_col in age_map.keys():
        ohca_signal.append(df[age_col] * w_a[age_map[age_col]])
    df['ohca_signal'] = sum(ohca_signal)
    
    # Anchor to total observed OHCA
    k = 3670 / df['ohca_signal'].sum()
    df['est_ohca'] = df['ohca_signal'] * k
    df_comarques[key]['ohca rel'] = df['est_ohca'] / df['pob'] * 100000

    cols_to_keep = [
        'comarca',
        'num_defi',
        'num_hospi',
        'pob',
        'geometry',
        'Total rel',
        'cardiac rel',
        'ohca rel'
    ]
    df_comarques[key] = df_comarques[key][cols_to_keep].copy()
    
df_comarques_total = df_comarques["total"]
df_comarques_women = df_comarques["women"]
df_comarques_men   = df_comarques["men"]

df_comarques_total


,comarca,num_defi,num_hospi,pob,geometry,Total rel,cardiac rel,ohca rel
0,alt camp,64.0,17.0,46388,"POLYGON ((1.23383 41.20393, 1.23383 41.20472, ...",192.59,43.097884,42.744349
1,alt emporda,283.0,49.0,148732,"MULTIPOLYGON (((2.997 42.13316, 2.99723 42.132...",211.23,47.269152,47.207904
2,alt penedes,236.0,22.0,114189,"POLYGON ((1.69316 41.2768, 1.69315 41.27542, 1...",188.78,42.245280,42.041801
3,alt urgell,53.0,21.0,21128,"MULTIPOLYGON (((1.32159 41.98664, 1.32235 41.9...",325.53,72.847262,72.550613
4,alta ribagorca,15.0,5.0,4040,"POLYGON ((0.85715 42.45393, 0.87114 42.44601, ...",173.70,38.870671,38.532079
5,anoia,443.0,24.0,128432,"POLYGON ((1.60696 41.51295, 1.60717 41.51292, ...",208.44,46.644805,46.335746
6,aran,31.0,8.0,10545,"POLYGON ((0.68006 42.72327, 0.68002 42.72329, ...",142.40,31.866341,31.416353
7,bages,257.0,35.0,185352,"POLYGON ((1.81103 41.60503, 1.81002 41.6048, 1...",259.34,58.035232,58.151399
8,baix camp,218.0,36.0,204458,"MULTIPOLYGON (((0.93153 40.99126, 0.93134 40.9...",183.19,40.994348,40.289950
9,baix ebre,134.0,19.0,82399,"MULTIPOLYGON (((0.44356 40.73713, 0.44377 40.7...",285.97,63.994506,64.071003


In [101]:
"""
We compute the AED coverage per 100.000k inhabitants. Important:
· Consider only AED locations, very different of being in a hospital from behind at home
· Cardiac arrest data was for Out of Hospital, so much better rationale.
"""
def coverage_calculation(df, pop = "pob", num = "num_defi", cov = "coverage_desfi"):
    df[cov] = np.where(
        df[pop] > 0,
        (df[num]) / df[pop] * 100000,
        0
    )
    return df


for key, df in df_comarques.items():

    # Convert columns to numeric
    df['num_defi']   = pd.to_numeric(df['num_defi'], errors='coerce').fillna(0)
    df['num_hospi']  = pd.to_numeric(df['num_hospi'], errors='coerce').fillna(0)
    df['pob']        = pd.to_numeric(df['pob'], errors='coerce').fillna(0)
    df['Total rel']  = pd.to_numeric(df['Total rel'], errors='coerce').fillna(0)

    # Compute coverage per 100k
    df = coverage_calculation(df, pop = "pob", num = "num_defi", cov = "coverage_desfi")
    df = coverage_calculation(df, pop = "pob", num = "num_hospi", cov = "coverage_hospi")
    df_comarques[key] = df

In [104]:
"""
We compute the normalised values for each column per comarca. This value is first centralised with the mean,
and then normalised to a maximum. This way, all variables have no units and are better plotted one with 
the other. 

The zero mask is no longer needed. 
"""

def normalise_zscore(df_comarques):
    for key, df in df_mort_com.items():
    
        zeros = df[cause] == 0
        non_zeros = ~zeros
    
        # Mean only over non-zero values
        mean_cause = df.loc[non_zeros, cause].mean()
        std_cause  = df.loc[non_zeros, cause].std()
    
        df[f'{cause} norm'] = 0.0
        if std_cause != 0:
            df.loc[non_zeros, f'{cause} norm'] = (df.loc[non_zeros, cause] - mean_cause) / std_cause
    
        # ---- COVERAGE NORM ----
        mean_cov = df['coverage_per_100k'].mean()
        std_cov  = df['coverage_per_100k'].std()
    
        df['coverage_norm'] = 0.0
        if std_cov != 0:
            df['coverage_norm'] = (df['coverage_per_100k'] - mean_cov) / std_cov
    
        df_comarques[key] = df
    return df_comarques



def normalize_percentile_trim(df, col, zero_mask=None, low_pct=0, high_pct=100):

    if zero_mask is None:
        zero_mask = (df[col] == 0)

    nonzero_mask = ~np.asarray(zero_mask)
    values = df.loc[nonzero_mask, col].astype(float).values

    if values.size < 3:
        raise ValueError("Need at least 3 non-zero values for percentile trimming.")

    low = np.percentile(values, low_pct)
    high = np.percentile(values, high_pct)
    if low == high:
        raise ValueError("Percentile bounds collapsed to a single value. Choose different percentiles.")

    inside_mask = (values >= low) & (values <= high)
    if not inside_mask.any():
        raise ValueError("No values lie inside the chosen percentile bounds.")

    mean_inside = values[inside_mask].mean()
    centered = values - mean_inside
    low_adj = low - mean_inside
    high_adj = high - mean_inside

    scale = max(abs(low_adj), abs(high_adj))
    if scale == 0:
        scale = 1e-9
    normalized = np.clip(centered / scale, -1, 1)
    norm_col = f"{col}_norm"
    df[norm_col] = 0.0
    df.loc[nonzero_mask, norm_col] = normalized

    return df

def normalise_maxabs(df_comarques, cause='cardiac rel', ohca = 'ohca rel', desfi='coverage_desfi', hospi = 'coverage_hospi', pop='pob'):
    for key, df in df_comarques.items():
        
        zeros = df[cause] == 0
        non_zeros = ~zeros

        df = normalize_percentile_trim(df, cause , zero_mask = zeros)
        df = normalize_percentile_trim(df, ohca , zero_mask = zeros)
        df = normalize_percentile_trim(df, pop , zero_mask = zeros, high_pct = 86)
        df = normalize_percentile_trim(df, desfi, zero_mask = zeros, high_pct = 95)
        df = normalize_percentile_trim(df, hospi, zero_mask = zeros, high_pct = 95)
        
        return df_comarques


df_comarques = normalise_maxabs(df_comarques)


df_comarques_total



,comarca,num_defi,num_hospi,pob,geometry,Total rel,cardiac rel,ohca rel,coverage_desfi,coverage_hospi,cardiac rel_norm,ohca rel_norm,pob_norm,coverage_desfi_norm,coverage_hospi_norm
0,alt camp,64.0,17.0,46388,"POLYGON ((1.23383 41.20393, 1.23383 41.20472, ...",192.59,43.097884,42.744349,137.966716,36.647409,-0.298586,-0.303070,-0.176148,-0.330365,0.011490
1,alt emporda,283.0,49.0,148732,"MULTIPOLYGON (((2.997 42.13316, 2.99723 42.132...",211.23,47.269152,47.207904,190.275126,32.945163,-0.177479,-0.174165,0.469779,0.007044,-0.031116
2,alt penedes,236.0,22.0,114189,"POLYGON ((1.69316 41.2768, 1.69315 41.27542, 1...",188.78,42.245280,42.041801,206.674899,19.266304,-0.323340,-0.323359,0.251766,0.112829,-0.188534
3,alt urgell,53.0,21.0,21128,"MULTIPOLYGON (((1.32159 41.98664, 1.32235 41.9...",325.53,72.847262,72.550613,250.851950,99.394169,0.565146,0.557721,-0.335573,0.397787,0.733589
4,alta ribagorca,15.0,5.0,4040,"POLYGON ((0.85715 42.45393, 0.87114 42.44601, ...",173.70,38.870671,38.532079,371.287129,123.762376,-0.421317,-0.424718,-0.443421,1.000000,1.000000
5,anoia,443.0,24.0,128432,"POLYGON ((1.60696 41.51295, 1.60717 41.51292, ...",208.44,46.644805,46.335746,344.929613,18.686932,-0.195606,-0.199352,0.341659,1.000000,-0.195201
6,aran,31.0,8.0,10545,"POLYGON ((0.68006 42.72327, 0.68002 42.72329, ...",142.40,31.866341,31.416353,293.978189,75.865339,-0.624678,-0.630217,-0.402366,0.675968,0.462816
7,bages,257.0,35.0,185352,"POLYGON ((1.81103 41.60503, 1.81002 41.6048, 1...",259.34,58.035232,58.151399,138.655099,18.882990,0.135099,0.141878,0.700900,-0.325924,-0.192945
8,baix camp,218.0,36.0,204458,"MULTIPOLYGON (((0.93153 40.99126, 0.93134 40.9...",183.19,40.994348,40.289950,106.623365,17.607528,-0.359659,-0.373952,0.821484,-0.532541,-0.207623
9,baix ebre,134.0,19.0,82399,"MULTIPOLYGON (((0.44356 40.73713, 0.44377 40.7...",285.97,63.994506,64.071003,162.623333,23.058532,0.308118,0.312834,0.051129,-0.171320,-0.144892


In [105]:
"""
We save our files, so we can later import them and plot multiple things.
"""
df_comarques_total.to_file("./Data/df_comarques_total.gpkg", driver="GPKG")
df_comarques_men.to_file("./Data/df_comarques_men.gpkg", driver="GPKG")
df_comarques_women.to_file("./Data/df_comarques_women.gpkg", driver="GPKG")
